In [8]:
import anthropic
from conf.settings import Settings
import json

In [2]:
settings = Settings()

In [32]:
class ToolAgent:
    def __init__(self,api_key: str):
        self.client = anthropic.Anthropic(api_key=api_key)
        self.model = settings.tool__agent_model
        self.max_tokens = settings.tool__agent_max_tokens
        self.temperature = settings.tool__agent_temperature
        self.knowledge_prompt = settings.tool__agent_prompt

        

    def _call_anthropic(self, query: str) -> str:
        response = self.client.messages.create(
            model=self.model,
            max_tokens=self.max_tokens,
            temperature=self.temperature,
            messages=[
                {"role": "user", "content": query}
            ]
        )
        return response.content[0].text

    def decide_tool(self, query: str) -> str:
        llm_input = self.knowledge_prompt.format(query=query)
        raw = self._call_anthropic(llm_input)
        try:
            data = json.loads(raw)        
            return data
        except json.JSONDecodeError:
            print("[WARN] Failed to parse tool selection response:", raw)
            return "none"

In [33]:
API_KEY = "".strip() 
tool_agent = ToolAgent(api_key=API_KEY)

user_query = "Tell me what to do in France in September"
response = tool_agent.decide_tool(user_query)

In [34]:
response

{'tool': 'none'}

In [ ]:
"get_weather" → para consultas tipo: "What's the weather in Paris next week?"

"make_reservation" → para: "Can you book a visit to the Louvre on Friday?"

"none" → para: "Tell me what to do in France in September."